In [51]:
import codecs
import os
# import re

# import pandas as pd
from _pickle import dump
import numpy as np

In [5]:
os.chdir('./miniTrainData')

In [6]:
# data=pd.DataFrame(columns=['Text']) 
data = []
for folders in os.listdir(os.getcwd()):
    cnt=0
    for file in os.listdir(folders):
        data.append((codecs.open('./'+folders+'/'+file,encoding='utf8',errors='replace').read()))
        cnt+=1
    print(folders,cnt)

sport 20
entertainment 20
business 20
politics 20
tech 20


In [28]:
data[10]

'Collins to compete in Birmingham\n\nWorld and Commonwealth 100m champion Kim Collins will compete in the 60m at the Norwich Union Grand Prix in Birmingham on 18 February.\n\nThe St Kitts and Nevis star joins British Olympic relay gold medallists Jason Gardener and Mark Lewis-Francis. Sydney Olympic 100m champion and world indoor record holder Maurice Greene and Athens Olympic 100m silver medallist Francis Obikwelu will also take part. Collins ran in Birmingham at the 2003 World Indoor Championships. "I\'m looking forward to competing against such a strong field," he said. "I got a great reception form the crowd at the NIA when I won my 60m world indoor silver medal in 2003 and it will be really exciting to return to this venue." The world champion says he\'s in good shape but he isn\'t underestimating the home competition. "Jason Gardener and Mark Lewis-Francis are Olympic gold medallists now and I\'m sure they\'ll be aiming to win in front of their home supporters. "I\'m looking forw

In [35]:
head_lines = [ 'startseq '+ doc.splitlines()[0] +' endseq' for doc in data]
Text = [ 'startseq '+' '.join(doc.splitlines()[1:])+' endseq' for doc in data]

In [36]:
head_lines[10]

'startseq Collins to compete in Birmingham endseq'

In [37]:
Text[10]

'startseq  World and Commonwealth 100m champion Kim Collins will compete in the 60m at the Norwich Union Grand Prix in Birmingham on 18 February.  The St Kitts and Nevis star joins British Olympic relay gold medallists Jason Gardener and Mark Lewis-Francis. Sydney Olympic 100m champion and world indoor record holder Maurice Greene and Athens Olympic 100m silver medallist Francis Obikwelu will also take part. Collins ran in Birmingham at the 2003 World Indoor Championships. "I\'m looking forward to competing against such a strong field," he said. "I got a great reception form the crowd at the NIA when I won my 60m world indoor silver medal in 2003 and it will be really exciting to return to this venue." The world champion says he\'s in good shape but he isn\'t underestimating the home competition. "Jason Gardener and Mark Lewis-Francis are Olympic gold medallists now and I\'m sure they\'ll be aiming to win in front of their home supporters. "I\'m looking forward to competing against Bri

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

/home/aninda/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [39]:
tokenizer=Tokenizer(filters='!"#$%&()*+,\'-./:;<=>?@[\\]^_`{|}~\t\n')

In [40]:
tokenizer.fit_on_texts(head_lines+Text)

In [41]:
word_to_index=tokenizer.word_index
index_to_word={ index:word for word,index in word_to_index.items()}
vocab_size=len(word_to_index)
print(vocab_size)

6221


In [42]:
train_sequence_text=tokenizer.texts_to_sequences(Text)
train_sequence_head=tokenizer.texts_to_sequences(head_lines)

In [43]:
maxlen_x=max(len(i) for i in train_sequence_text)
maxlen_y=max(len(i) for i in train_sequence_head)
print(maxlen_x,maxlen_y)

1247 9


In [47]:
from keras.utils import to_categorical

In [48]:
def data_for_model(sequences_head,sequences_text):
    x1=[]
    x2=[]
    y=[]
    for sequence_idx,sequence in enumerate(sequences_head):
        for idx in range(1,len(sequence)):
            in_seq,out_seq=sequence[:idx],sequence[idx]
            in_seq=pad_sequences([in_seq],maxlen=maxlen_y)[0]
            out_seq=to_categorical([out_seq],num_classes=vocab_size+1)[0]
            x1.append(pad_sequences([sequences_text[sequence_idx]],maxlen=maxlen_x)[0])
            x2.append(in_seq)
            y.append(out_seq)
    return np.array(x1),np.array(x2),np.array(y)

In [49]:
x1_train,x2_train,y_train=data_for_model(train_sequence_head,train_sequence_text)

In [53]:
os.chdir('..')

In [54]:
embedding_dim=200
embedding_weights = np.empty((400000,embedding_dim))

In [55]:
i=0
glove_word_to_index = {}
with open('glove.6B.200d.txt','r') as file:
    for line in file:
        token=line.strip().split()
        glove_word_to_index[token[0]] = i
        embedding_weights[i]=list(map(float,token[1:]))

        i+=1

file.close()

In [56]:
np.random.seed(5)
embedding_weights*=.1

In [57]:
shape=(vocab_size+1,embedding_dim)
scale = embedding_weights.std()*np.sqrt(12)/2
print(scale)
embedding = np.random.uniform(low=-scale, high=scale, size=shape)

0.0661404484678092


In [58]:
word_found_in_glove = 0
for word,idx in word_to_index.items():
    if word in glove_word_to_index:
        embedding[idx] = embedding_weights[glove_word_to_index[word]]
        word_found_in_glove += 1
        
print(word_found_in_glove)

6073


In [59]:
dump(x1_train, open('x1_train.pkl', 'wb'))
dump(x2_train, open('x2_train.pkl', 'wb'))
dump(y_train, open('y_train.pkl', 'wb'))
dump(embedding, open('embedding.pkl', 'wb'))